In [1]:
import requests
import os
import pandas as pd

direc = 'C:/Users/rvozzi/genommalabinternacional/Noelia Soledad Riccelli - Plan BI/Nielsen'
file_name = '/Base Nielsen Corporales Jun 22.xlsx'

# Cargo la tabla de la Hoja 1
tabla_u = pd.read_excel(io = direc + file_name, sheet_name = 'Hoja1', header = 5, usecols = "A:X")
tabla_m = pd.read_excel(io = direc + file_name, sheet_name = 'Hoja1', header = 5, usecols = "A:E,Y:AQ")

def procesar_tabla(tabla, tipo):
    
    # Saco las filas que tengan null ya que son totales y tambien las SDESC == 'TOTAL ARGENTINA - INC+AS'
    tabla = tabla.dropna()
    tabla.columns = [x.replace('.1', '') for x in tabla.columns]
    tabla = tabla[~tabla['SDESC'].isin(['TOTAL ARGENTINA - INC+AS'])]

    #Hago un melt de la tabla
    meses = ['ENE', 'MAR', 'MAY', 'JUL', 'SEP', 'NOV']
    meses_num = ['01', '03', '05', '07', '09', '11']
    id_vars = ['SDESC', 'FABRICANTE ARM', 'MARCA ARM', 'BRAND', 'LDESC']
    final_vars = ['Canal', 'Fabricante', 'Marca', 'Brand', 'Descripcion']
    value_vars = ['MAY/JUN 2019', 'JUL/AGO 2019', 'SEP/OCT 2019', 'NOV/DIC 2019',
                  'ENE/FEB 2020', 'MAR/ABR 2020', 'MAY/JUN 2020', 'JUL/AGO 2020',
                  'SEP/OCT 2020', 'NOV/DIC 2020', 'ENE/FEB 2021', 'MAR/ABR 2021',
                  'MAY/JUN 2021', 'JUL/AGO 2021', 'SEP/OCT 2021', 'NOV/DIC 2021',
                  'ENE/FEB 2022', 'MAR/ABR 2022', 'MAY/JUN 2022']

    tabla = pd.melt(tabla, 
                      id_vars = id_vars,
                      value_vars = value_vars,
                      value_name = tipo, var_name = 'Año_Mes')

    #Transformo los bimestres a fechas
    tabla['Mes'] = tabla['Año_Mes'].map(lambda x: meses_num[meses.index(x.split('/')[0])])
    tabla['Año'] = tabla['Año_Mes'].map(lambda x: x.split()[1])
    tabla['Año_Mes'] = tabla['Año'].map(str) + '-' + tabla['Mes']

    #Renombro las columnas
    tabla.rename(dict(zip(id_vars,final_vars)), axis=1, inplace=True)
    tabla = tabla[ final_vars + ['Año_Mes', tipo]]

    return tabla

In [2]:
tabla = pd.merge(procesar_tabla(tabla_u, 'Cantidad'), 
                 procesar_tabla(tabla_m, 'Monto'),
                 left_on = ['Canal', 'Fabricante', 'Marca', 'Brand', 'Descripcion', 'Año_Mes'],
                 right_on = ['Canal', 'Fabricante', 'Marca', 'Brand', 'Descripcion', 'Año_Mes'])

tabla.to_csv(path_or_buf = direc + '/base.csv' ,sep = ',', index = False)

In [3]:
#estas son el segmento de cremas corporales, faltan los otros segmentos

tabla

,Canal,Fabricante,Marca,Brand,Descripcion,Año_Mes,Cantidad,Monto
0,ARGENTINA FARMA,M PROPIAS,M PROPIAS,M PROPIAS,M PROPIAS M PROPIAS M PROPIAS,2019-05,70399.72,3250444.75
1,ARGENTINA FARMA,M PROPIAS,M PROPIAS,O MARCAS,M PROPIAS M PROPIAS O MARCAS,2019-05,287.21,23351.32
2,ARGENTINA FARMA,ANTONIO PUIG,212,O MARCAS,ANTONIO PUIG 212 O MARCAS,2019-05,105.87,38487.72
3,ARGENTINA FARMA,ANTONIO PUIG,ADOLFO DOMINGUEZ,O MARCAS,ANTONIO PUIG ADOLFO DOMINGUEZ O MARCAS,2019-05,99.40,13748.01
4,ARGENTINA FARMA,ANTONIO PUIG,BENETTON,O MARCAS,ANTONIO PUIG BENETTON O MARCAS,2019-05,70.57,10577.20
...,...,...,...,...,...,...,...,...
14131,ARGENTINA SELF INDEPENDIENTE,UNILEVER,ST. IVES,ST.IVES PIEL RENOVADA,UNILEVER ST. IVES ST.IVES PIEL RENOVADA,2022-05,411.77,184981.58
14132,ARGENTINA SELF INDEPENDIENTE,UNILEVER,ST. IVES,ST.IVES REAFIRMANTE,UNILEVER ST. IVES ST.IVES REAFIRMANTE,2022-05,87.46,52853.77
14133,ARGENTINA SELF INDEPENDIENTE,UNILEVER,VASELINE,VASELINE I.C. CELULITIS CONTRO,UNILEVER VASELINE VASELINE I.C. CELULITIS CONTRO,2022-05,0.00,0.00
14134,ARGENTINA SELF INDEPENDIENTE,UNILEVER,VASELINE,VASENOL,UNILEVER VASELINE VASENOL,2022-05,0.00,0.00
